In [1]:
import gensim
from gensim.models.wrappers import FastText
import time
import os
from tkinter import filedialog
from tkinter import *
import tkinter.messagebox
import re
from collections import Counter

In [2]:
start = time.time()
model = FastText.load_fasttext_format(r"C:\Users\AlnthraevaN\Documents\Skripsi\cc.en.300.bin")
print(f"Model loaded in {time.time()-start}s")

Model loaded in 170.78221154212952s


In [4]:
words = list(model.wv.vocab)

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
    
WORDS = w_rank

In [62]:
def words(text): return re.findall(r'\w+', text.lower())

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def browse_button():
    directory = filedialog.askdirectory()
    path.set(directory)
    tb2.insert(0.0, str(directory))
    #print("1 " + directory)

def retrieve_input():
    input = tb1.get("1.0","end-1c")
    return input

def retrieve_directory():
    directory = tb2.get("1.0", "end-1c")
    return directory

def resultwindow(fn):
    resultwindow = Toplevel(root)
    resultwindow.geometry("350x500")
    resultwindow.title("QWERY")
    resultwindow.resizable(width=False, height=False)
    appname = Label(resultwindow, text="QWERY Search Engine")
    appname.pack()
    lb = Listbox(resultwindow, height=10, width=50)
    lb.pack()
#     sb = Scrollbar(lb)
#     sb.pack(side=RIGHT, fill=BOTH)
#     listbox.config(yscrollcommand = sb.set) 
#     scrollbar.config(command = listbox.yview) 
    print(fn)
    for i in fn:
        lb.insert(END, i)
    
# r=root, d=directories, f = files
def search(path):
    if(os.path.exists(path)):
        word = str(retrieve_input()) #Get Input from TKinter
        kw = [] #Array of keyword(s) to search
        kw.append(str.lower(word))
        keyword = model.most_similar(positive=[word], topn=100)
        #print(keyword)
        #Spell checking if the first NN has score lower than 70 (Probability a typo on input)
        if(keyword[0][1] < 0.70):
            if(word not in str.lower(correction(word))):
                word = str.lower(correction(word))
                kw.append(word)
                keyword = model.most_similar(positive=[word], topn=100)
        filename = []
        filepath = []
        kwused = []
        #Lists of file found with keyword
        for i in keyword:
            if(i[1] >= 0.60):
                FLAG = TRUE
                #print("Testing " + str.lower(i[0]))
                #Checking if NN is in kw array
                for l in kw:
                    if(l in str.lower(i[0])):
                        FLAG = FALSE
                if(FLAG and str.lower(i[0]) not in kw):
                    #print(str.lower(i[0]) + " added to the list")
                    kw.append(str.lower(i[0]))
        for root, dirs, files in os.walk(path):
            # Searching...
            for j in files:
                for k in kw:
                    if str.lower(k) in str.lower(os.path.splitext(j)[0]):
                        #lists.append(root + "\\" + j)
                        filepath.append(root)
                        filename.append(j)
                        kwused.append(str.lower(k))
    # Search the NN of keyword
    #         for i in keyword:
    #             if(i[0] > 0.60):
    #                 print("Testing " + str.lower(i[1]))
    #                 for l in kw:
    #                     if(l not in str.lower(i[1]) and str.lower(i[1]) not in kw):
    #                         print(kw)
    #                         kw.append(str.lower(i[1]))
    #                         print("Searching " + str.lower(i[1]))
    #                         for k in files:
    #                             if str.lower(i[1]) in str.lower(os.path.splitext(k)[0]):
    #                                 print(str(i[1]))
    #                                 if root + "/" + k not in lists:
    #                                     print("got " + str(i[1]))
    #                                     lists.append(root + "/" + k)

        if filename:
            for i in 1, 2, 3:
                
#             print(filepath)
#             print(filename)
#             print(kwused)
            resultwindow(filename)
        else:
             tkinter.messagebox.showerror(title="No File Found", message="No File Found")
    else:
        tkinter.messagebox.showerror(title="Directory Not Found", message="Please enter the right directory")

In [63]:
root = Tk()
path = StringVar()

root.geometry("350x200")
root.title("QWERY")
root.resizable(width=False, height=False)
appname = Label(root, text="QWERY Search Engine")
appname.grid(row=0, columnspan=4, padx=(20,20), pady=(10,20))
lbl1 = Label(root, text="Keyword: ")
lbl1.grid(row=2, column=0, padx=(20,0))
tb1 = Text(master=root, height=1, width=30, wrap="none")
#e1.insert(0, 'keyword')
tb1.grid(row=2, column=1, columnspan=2, padx=(0,20))
tb2 = Text(master=root, height=1, width=30) #, textvariable=path
tb2.grid(row=3, column=0, columnspan=2, pady=(10,0), padx=(22,0))
button2 = Button(master= root, text="Browse", command=browse_button)
button2.grid(row=3, column=2, padx=(8, 20), pady=(10,0))
button1 = Button(master=root, text="Search", width=30, command= lambda: search(retrieve_directory()))
button1.grid(row=4, columnspan=4, pady=(10,0))

root.mainloop()

['Bag of Tricks for Efficient Text Classification.pdf', 'npy_1_7_deprecated_api.h', 'npy_no_deprecated_api.h', 'test_deprecations.py', 'test_deprecations.cpython-37.pyc', 'test_deprecations.py', 'test_deprecations.cpython-37.pyc', 'test_deprecations.py', 'test_deprecations.cpython-37.pyc', 'locations.py', 'deprecation.py', 'locators.py']


In [30]:
correction("cay")

'cay'